<a href="https://colab.research.google.com/github/kuberiitb/learnings/blob/master/pytorch/lecture_07_linear_regression_in_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Learnings from [Abhishek Thakur Pytoch Series](https://www.youtube.com/watch?v=_R-mvKBD5U8&list=PL98nY_tJQXZln8spB5uTZdKN08mYGkOf2&index=7)

In [2]:
import torch 

In [3]:
torch.cuda.is_available()

True

In [4]:
!nvidia-smi

Thu Feb  3 09:40:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## For tablular data 

# Dataloader class
Create a class

using dataset class

Create from scratch

## Contents of class
## tabular data, 2 D
__init__: data, targets

__len__: 

__getitems__: idx, retrun dict of tensors with keys sample, target



In [5]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [6]:
class CustomDataSet:
  def __init__(self, data, targets):
    self.data = data
    self.targets = targets

  def __len__(self):
    return self.data.shape[0]
  
  def __getitem__(self, idx):
    current_sample = self.data[idx]
    current_target = self.targets[idx]

    return {
        "x": torch.tensor(current_sample, dtype = torch.float), 
        "y": torch.tensor(current_target, dtype = torch.long),
    }

In [7]:
# default n_samples=100, n_features=20, n_classes=2
# returns x, y
data, targets = make_classification(n_samples=1000)

In [8]:
train_data, test_data, train_targets, test_targets =  train_test_split(data, targets, stratify=targets)

In [9]:
print(train_data.shape, 
      test_data.shape, 
      train_targets.shape, 
      test_targets.shape)

(750, 20) (250, 20) (750,) (250,)


In [10]:
train_dataset = CustomDataSet(data=train_data,  targets=train_targets)
test_dataset = CustomDataSet(data=test_data,  targets=test_targets)

In [11]:
len(train_dataset)

750

In [12]:
train_dataset[0]

{'x': tensor([ 0.4453, -1.3352, -0.1936,  0.3496,  1.1778, -0.7850,  0.2890, -1.8016,
         -2.4640,  0.8013, -0.3833,  0.6848, -2.2016, -1.4477,  0.6421, -0.3431,
         -1.9471,  0.9996,  0.0369, -0.4622]), 'y': tensor(0)}

In [16]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=4, num_workers=2)

## Build Model

In [32]:
def model(x, W, b):
  return torch.matmul(x, W) +  b

W = torch.randn(20, 1, requires_grad=True)
b = torch.randn(1, requires_grad=True)

learning_rate = 0.001
N_EPOCHS = 20

for epoch in range(N_EPOCHS):
  epoch_loss = 0
  counter = 0
  for data in train_loader:
    xtrain, ytrain = data["x"], data["y"]
    output = model(xtrain, W, b)
    if counter==50:
      print("before", b.grad)
    
    loss = torch.mean((ytrain.view(-1) - output.view(-1))**2)
    epoch_loss = epoch_loss + loss.item()
    loss.backward()
    if counter==50:
      print("after", b.grad)
    with torch.no_grad():
      W = W - learning_rate * W.grad
      b = b - learning_rate * b.grad
    counter += 1
    W.requires_grad_(True)
    b.requires_grad_(True)
  print(epoch, epoch_loss/counter, loss )
    

before None
after tensor([-2.7850])
0 6.791921442612372 tensor(1.6339, grad_fn=<MeanBackward0>)
before None
after tensor([-1.3908])
1 2.7665401164521564 tensor(0.6196, grad_fn=<MeanBackward0>)
before None
after tensor([-1.0033])
2 1.3764881298263023 tensor(0.2007, grad_fn=<MeanBackward0>)
before None
after tensor([-0.8452])
3 0.7279280949225451 tensor(0.0602, grad_fn=<MeanBackward0>)
before None
after tensor([-0.7677])
4 0.4172610244565108 tensor(0.0294, grad_fn=<MeanBackward0>)
before None
after tensor([-0.7290])
5 0.26583013630551383 tensor(0.0354, grad_fn=<MeanBackward0>)
before None
after tensor([-0.7112])
6 0.19072603287690498 tensor(0.0501, grad_fn=<MeanBackward0>)
before None
after tensor([-0.7050])
7 0.15285979969268468 tensor(0.0642, grad_fn=<MeanBackward0>)
before None
after tensor([-0.7048])
8 0.13347481732613387 tensor(0.0751, grad_fn=<MeanBackward0>)
before None
after tensor([-0.7077])
9 0.12341071810186939 tensor(0.0828, grad_fn=<MeanBackward0>)
before None
after tensor([

In [36]:
train_targets[0:10]

array([0, 1, 0, 1, 1, 0, 1, 1, 1, 0])

In [61]:
labels = []
outputs = []
for data in test_loader:
  xtest, ytest = data["x"], data["y"]
  with torch.no_grad():
    output = model(xtest, W, b)
  outputs.append(output)
  labels.append(ytest)
  
print(len(labels), len(outputs))

63 63


In [62]:
print(labels[:3]) # batchwise output
print(len(labels)) # same as batchsize


[tensor([1, 0, 1, 1]), tensor([1, 1, 0, 1]), tensor([0, 1, 1, 0])]
63


In [63]:
labels_cat = torch.cat(labels) # flattening

print(labels_cat[:10])
print(len(labels_cat))

labels_cat.view(-1) # making vector

tensor([1, 0, 1, 1, 1, 1, 0, 1, 0, 1])
250


tensor([1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0,
        1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0,
        0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0,
        1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1,
        0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1,
        0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1,
        1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1,
        0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
        0, 1, 1, 0, 1, 0, 1, 1, 0, 1])

In [75]:
outputs_cat = torch.cat(outputs) # flattening

print(outputs_cat[:10])
print(len(outputs_cat))

outputs_cat.view(-1)[:20] # making vector

tensor([[0.3905],
        [0.1541],
        [1.4401],
        [0.0017],
        [0.7410],
        [1.2124],
        [0.2074],
        [0.7838],
        [0.4497],
        [0.7517]])
250


tensor([ 0.3905,  0.1541,  1.4401,  0.0017,  0.7410,  1.2124,  0.2074,  0.7838,
         0.4497,  0.7517,  1.4073,  0.2446,  0.7040,  0.4037,  1.1659,  1.0694,
         0.7377, -0.0165,  1.0258, -0.0451])

In [72]:
from sklearn.metrics import roc_auc_score
roc_auc_score(torch.cat(labels).view(-1) , torch.cat(outputs).view(-1))

0.8969976313936369

# Issues: use sigmoid for classification problems

In [74]:
model

<function __main__.model>